## Econ 430
### UCLA, Master of Quantitative Economics
#### Dr. Randall R. Rojas
Note: For more details on the codes, please see our textbook; Using Python for Introductory Econometrics by F. Heiss & D. Brunner.

# 1) Two Stage Least Squares Estimation
## Example: Labor Supply of Married,  Working Women

In [1]:
import wooldridge as woo
import numpy as np
import pandas as pd
import linearmodels.iv as iv

mroz = woo.dataWoo('mroz')

# restrict to non-missing wage observations:
mroz = mroz.dropna(subset=['lwage'])

# 2SLS regressions:
reg_iv1 = iv.IV2SLS.from_formula(
    'hours ~ 1 + educ +  age + kidslt6 + nwifeinc +'
    '[np.log(wage) ~ exper + I(exper**2)]', data=mroz)
results_iv1 = reg_iv1.fit(cov_type='unadjusted', debiased=True)

reg_iv2 = iv.IV2SLS.from_formula(
    'np.log(wage) ~ 1 + educ + exper + I(exper**2) +'
    '[hours ~ age + kidslt6 + nwifeinc]', data=mroz)
results_iv2 = reg_iv2.fit(cov_type='unadjusted', debiased=True)

# print results:
table_iv1 = pd.DataFrame({'b': round(results_iv1.params, 4),
                          'se': round(results_iv1.std_errors, 4),
                          't': round(results_iv1.tstats, 4),
                          'pval': round(results_iv1.pvalues, 4)})
print(f'table_iv1: \n{table_iv1}\n')

table_iv2 = pd.DataFrame({'b': round(results_iv2.params, 4),
                          'se': round(results_iv2.std_errors, 4),
                          't': round(results_iv2.tstats, 4),
                          'pval': round(results_iv2.pvalues, 4)})
print(f'table_iv2: \n{table_iv2}\n')

cor_u1u2 = np.corrcoef(results_iv1.resids, results_iv2.resids)[0, 1]
print(f'cor_u1u2: {cor_u1u2}\n')

table_iv1: 
                      b        se       t    pval
Intercept     2225.6618  574.5641  3.8737  0.0001
educ          -183.7513   59.0998 -3.1092  0.0020
age             -7.8061    9.3780 -0.8324  0.4057
kidslt6       -198.1543  182.9291 -1.0832  0.2793
nwifeinc       -10.1696    6.6147 -1.5374  0.1249
np.log(wage)  1639.5556  470.5757  3.4841  0.0005

table_iv2: 
                    b      se       t    pval
Intercept     -0.6557  0.3378 -1.9412  0.0529
educ           0.1103  0.0155  7.1069  0.0000
exper          0.0346  0.0195  1.7742  0.0767
I(exper ** 2) -0.0007  0.0005 -1.5543  0.1209
hours          0.0001  0.0003  0.4945  0.6212

cor_u1u2: -0.9037694196299446



# 1) Three Stage Least Squares Estimation
## Example: Labor Supply of Married,  Working Women

In [2]:
import wooldridge as woo
import numpy as np
import linearmodels.system as iv3

mroz = woo.dataWoo('mroz')

# restrict to non-missing wage observations:
mroz = mroz.dropna(subset=['lwage'])

# 3SLS regressions:
formula = {'eq1': 'hours ~ 1 + educ + age + kidslt6 + nwifeinc +'
                  '[np.log(wage) ~ exper+I(exper**2)]',
           'eq2': 'np.log(wage) ~ 1 + educ + exper + I(exper**2) +'
                  '[hours ~ age + kidslt6 + nwifeinc]'}

reg_3sls = iv3.IV3SLS.from_formula(formula, data=mroz)

results_3sls = reg_3sls.fit(cov_type='unadjusted', debiased=True)
print(f'results_3sls: \n{results_3sls}\n')


results_3sls: 
                           System GLS Estimation Summary                           
Estimator:                        GLS   Overall R-squared:                  -2.3957
No. Equations.:                     2   McElroy's R-squared:                 0.7846
No. Observations:                 428   Judge's (OLS) R-squared:            -2.3957
Date:                Sun, Nov 07 2021   Berndt's R-squared:                  0.5181
Time:                        15:30:37   Dhrymes's R-squared:                -2.3957
                                        Cov. Estimator:                  unadjusted
                                        Num. Constraints:                      None
                    Equation: eq1, Dependent Variable: hours                    
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
Intercept        2305.9     511.54     4.5077     0.0000      1300.4  